In [2]:
import pandas as pd

FB_PATH = "../../artifact/core8/core8_06_fallback_decisions.csv"
fb = pd.read_csv(FB_PATH)

print(fb.columns.tolist())
fb.head()

['run_id', 'case_id', 'antibody_id', 'step', 'blocked_rate_window', 'veto_streak', 'action_toggle_rate', 'SoMS_cumsum_window', 'refusal_triggered', 'refusal_reason_code', 'cooldown_ok', 'block_s', 'veto_s', 'tog_s', 'soms_s', 'fallback_score', 'fallback_evaluated', 'fallback_entered', 'fallback_stage', 'fallback_reason_code', 'fallback_rule_id', 'fallback_score.1', 'is_hold', 'is_partial_seal', 'is_refusal', 'is_fallback']


,run_id,case_id,antibody_id,step,blocked_rate_window,veto_streak,action_toggle_rate,SoMS_cumsum_window,refusal_triggered,refusal_reason_code,...,fallback_evaluated,fallback_entered,fallback_stage,fallback_reason_code,fallback_rule_id,fallback_score.1,is_hold,is_partial_seal,is_refusal,is_fallback
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,0.0,0,0.0,0.0,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.000,True,False,False,False
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,0.0,0,0.0,0.6,False,REASON_MIN_STEPS_NOT_REACHED,...,True,False,HOLD,REASON_MIN_STEPS_NOT_REACHED,core8_06_v1,0.006,True,False,False,False


In [ ]:
import pandas as pd

FB_PATH = "../../artifact/core8/core8_06_fallback_decisions.csv"
fb = pd.read_csv(FB_PATH)

# ---- score 컬럼 확정 (fallback_score.1 같은 중복 방지) ----
score_col = "fallback_score"
if "fallback_score.1" in fb.columns:
    # 둘 중 하나만 쓰자: 기본은 fallback_score
    pass

# ---- 이벤트 정의 ----
fb["event_hard"] = (fb["refusal_triggered"] == True) | (fb["fallback_stage"] == "REFUSAL") | (fb["is_refusal"] == True)
fb["event_soft"] = (fb["fallback_stage"] == "PARTIAL_SEAL") | (fb["is_partial_seal"] == True)

hard = fb.loc[fb["event_hard"], score_col].dropna()
soft = fb.loc[fb["event_soft"], score_col].dropna()

print("hard event rows:", len(hard))
print("soft event rows:", len(soft))

assert len(hard) + len(soft) > 0, "❌ event(REFUSAL/PARTIAL_SEAL) 기반 데이터도 없음"

# ---- threshold 후보 ----
# hard가 있으면 hard 기준, 없으면 soft 기준으로 잡는다.
base = hard if len(hard) > 0 else soft
desc = base.describe()
print(desc)

threshold_candidates = {
    "aggressive": float(base.quantile(0.10)),   # 더 빨리 터지게
    "balanced":   float(base.quantile(0.25)),   # 데모용
    "conservative": float(base.quantile(0.50)), # 보수적
}

threshold_candidates

Columns: ['run_id', 'case_id', 'antibody_id', 'step', 'blocked_rate_window', 'veto_streak', 'action_toggle_rate', 'SoMS_cumsum_window', 'refusal_triggered', 'refusal_reason_code', 'cooldown_ok', 'block_s', 'veto_s', 'tog_s', 'soms_s', 'fallback_score', 'fallback_evaluated', 'fallback_entered', 'fallback_stage', 'fallback_reason_code', 'fallback_rule_id', 'fallback_score.1', 'is_hold', 'is_partial_seal', 'is_refusal', 'is_fallback']


AssertionError: ❌ fallback 발생 기록이 없음